In [13]:
import pandas as pd
import torch
import torch.nn as nn
from torch.utils.data import DataLoader, TensorDataset
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import numpy as np

Our data has 11191357 rows in total

In [14]:
df = pd.read_csv("data.csv")
df

/tmp/ipykernel_22342/429624954.py:1: DtypeWarning: Columns (11,12,14,17,18,19,20,22,23,24,25,26,27) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv("data.csv")


,FISCAL YEAR,FISCAL PERIOD,DEPARTMENT,DIVISION,VENDOR,BUDGET_REF,FUND_TYPE,FUND,FUND_DESCR,DEPTID,...,SCH_CODE_DESCR,PC_BU,PC_BU_DESCR,PROJECT,PROJECT_DESCR,ACTIVITY,ACTIVITY_DESCR,CHECK_NUMBER,CHECK_DATE,AMOUNT
0,2021,2,DEPT OF ELECTIONS,SUSSEX COUNTY ELECTIONS,-TAMERA J TIMMONS,2020.0,GENERAL,100,General Fund,700401,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0001050550,08/31/2020,322.50
1,2021,6,DEPT OF NATRL RES AND ENV CONT,PARKS AND RECREATION,.CHANCELLOR LANE LLC,2021.0,SPECIAL,100,General Fund,400302,...,NaN,40000.0,Dept of Natural Resources,C4032EM-MARINA,Emergency-Marina,CONSTRUCTN,Construction,0001812359,12/16/2020,16980.00
2,2021,6,DEPT OF NATRL RES AND ENV CONT,PARKS AND RECREATION,.CHANCELLOR LANE LLC,2021.0,SPECIAL,100,General Fund,400302,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0001812359,12/16/2020,10.00
3,2021,12,DEPT OF EDUCATION,CAMPUS COMMUNITY SCHOOL,1 WORLD GLOBES MAPS LLC,2021.0,SPECIAL,74,Campus Community Local,957400,...,Campus Community School,NaN,NaN,NaN,NaN,NaN,NaN,0001867325,06/24/2021,317.95
4,2021,6,DEPT OF STATE,DELAWARE ECONOMIC DEVELOPMENT,1007 MARKET FOOD HALL INC,2020.0,FEDERAL,225,Federal Funds,201001,...,NaN,10000.0,Executive,000000000015927,DOS Small Business De Relief,PUBLIC_AST,GM10-PUBLIC ASSISTANCE,0001091347,12/29/2020,36659.32
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11191352,2026,1,DEPT OF STATE,PUBLIC INTEGRITY COMMISSION,ZOOM VIDEO COMMUNICATIONS INC,2026.0,SPECIAL,100,General Fund,200108,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0000556931,07/15/2025,0.80
11191353,2026,1,DEPT OF STATE,PUBLIC INTEGRITY COMMISSION,ZOOM VIDEO COMMUNICATIONS INC,2026.0,GENERAL,100,General Fund,200108,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0000556931,07/15/2025,299.00
11191354,2026,1,JUDICIAL,JUSTICE OF THE PEACE COURT,ZUAVEY KADAFI,2026.0,SPECIAL,880,Custodial - Pass Thru,21310,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0000439727,07/11/2025,26.00
11191355,2026,1,DEPT OF LABOR,OFFICE OF WORKERS'COMPENSATION,ZURICH AMERICAN INS CO,2026.0,SPECIAL,100,General Fund,600701,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0000441778,07/18/2025,2866.52


In [8]:
!pip install -U sentence-transformers

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


Defaulting to user installation because normal site-packages is not writeable


In [15]:
from sentence_transformers import SentenceTransformer
import numpy as np

# Load model
textemb_model = SentenceTransformer('all-MiniLM-L6-v2',device ="cuda")
textemb_model = torch.nn.DataParallel(textemb_model)
#print(textemb_model.device)

# List of text columns to encode
text_columns = [
    'DEPARTMENT',
    'DIVISION',
    'FUND_DESCR',
    'DEPTID_DESCR',
    'APPROP_DESCR',
    'ACCOUNT_DESCR',
    'PROGRAM_DESCR'
]

# Dictionary to store embeddings
embeddings_dict = {}

for col in text_columns:
    # Handle nulls
    df[col] = df[col].fillna('missing')

    # Encode column
    embeddings = textemb_model.module.encode(
        df[col].tolist(),
        convert_to_numpy=True,
        batch_size=64,      # for speed on large datasets
        show_progress_bar=True,
        device=["cuda:0","cuda:1","cuda:2","cuda:3","cuda:4","cuda:5","cuda:6","cuda:7"]
    )

    embeddings_dict[col] = embeddings
    print(f"{col} -> shape {embeddings.shape}")


Chunks: 100%|██████████| 2239/2239 [05:37<00:00,  6.64it/s]


DEPARTMENT -> shape (11191357, 384)


Chunks: 100%|██████████| 2239/2239 [05:44<00:00,  6.50it/s]


DIVISION -> shape (11191357, 384)


Chunks: 100%|██████████| 2239/2239 [05:45<00:00,  6.48it/s]


...

APPROP_DESCR -> shape (11191357, 384)


Chunks: 100%|██████████| 2239/2239 [05:46<00:00,  6.46it/s]


ACCOUNT_DESCR -> shape (11191357, 384)


Chunks: 100%|██████████| 2239/2239 [05:51<00:00,  6.36it/s]


PROGRAM_DESCR -> shape (11191357, 384)


In [16]:
# Concatenate embeddings horizontally
all_text_embeddings = np.hstack([embeddings_dict[col] for col in text_columns])

In [17]:
# -------------------------------
# Encode FISCAL YEAR
# -------------------------------
# Fill missing with median year
df['fiscal_year'] = pd.to_numeric(df['FISCAL YEAR'], errors='coerce')
median_year = df['fiscal_year'].median()
df['fiscal_year'] = df['fiscal_year'].fillna(median_year)

# Standard scale
df['fiscal_year_scaled'] = (df['fiscal_year'] - df['fiscal_year'].mean()) / df['fiscal_year'].std()

# -------------------------------
# Encode FISCAL PERIOD (cyclical)
# -------------------------------
# Fill missing with median
median_period = df['FISCAL PERIOD'].median()
df['FISCAL PERIOD'] = df['FISCAL PERIOD'].fillna(median_period)

# Sin/Cos encoding
df['fp_sin'] = np.sin(2 * np.pi * df['FISCAL PERIOD'] / 12)
df['fp_cos'] = np.cos(2 * np.pi * df['FISCAL PERIOD'] / 12)

# -------------------------------
# Encode CHECK_DATE
# -------------------------------
# Convert to datetime
df['check_date'] = pd.to_datetime(df['CHECK_DATE'], errors='coerce')

# Fill missing dates with a sentinel (e.g., 1970-01-01)
df['check_date'] = df['check_date'].fillna(pd.Timestamp('1970-01-01'))

# Extract components
df['check_year'] = df['check_date'].dt.year
df['check_month'] = df['check_date'].dt.month
df['check_dayofweek'] = df['check_date'].dt.dayofweek

# Standard scale year
df['check_year_scaled'] = (df['check_year'] - df['check_year'].mean()) / df['check_year'].std()

# Cyclical encoding for month and day of week
df['check_month_sin'] = np.sin(2 * np.pi * df['check_month'] / 12)
df['check_month_cos'] = np.cos(2 * np.pi * df['check_month'] / 12)

df['check_dow_sin'] = np.sin(2 * np.pi * df['check_dayofweek'] / 7)
df['check_dow_cos'] = np.cos(2 * np.pi * df['check_dayofweek'] / 7)

In [19]:
import numpy as np

# Save the dictionary to a compressed .npz file
np.savez_compressed("embeddings_dict.npz", **embeddings_dict)


KeyboardInterrupt: 

In [18]:
# Ensure AMOUNT is numeric
df['amount'] = pd.to_numeric(df['AMOUNT'], errors='coerce')
df['amount'] = df['amount'].fillna(df['amount'].median())

# -------------------------------
# Remove rows with amount < 1
# -------------------------------
mask = df['amount'] >= 1
df = df[mask].reset_index(drop=True)

# Recompute numeric features and text embeddings after filtering if necessary
# (Assuming all_text_embeddings was computed before filtering, we need to filter it too)
all_text_embeddings = all_text_embeddings[mask.values]

# -------------------------------
# Combine all features
# -------------------------------

numeric_features = df[[ 
    'fiscal_year_scaled', 'fp_sin', 'fp_cos',
    'check_year_scaled', 'check_month_sin', 'check_month_cos',
    'check_dow_sin', 'check_dow_cos'
]].to_numpy()

# Final feature array
X = np.hstack([all_text_embeddings, numeric_features])

# Target
y = df['amount'].to_numpy().reshape(-1, 1)

# -------------------------------
# Apply log transform to y
# -------------------------------
y_log = np.log1p(y)  # log(1 + y) to handle small amounts safely

# Convert to tensors
X_tensor = torch.from_numpy(X).float()
y_tensor = torch.from_numpy(y_log).float()

# Train-test split
from torch.utils.data import TensorDataset, DataLoader
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(
    X_tensor, y_tensor, test_size=0.2, random_state=42
)

train_ds = TensorDataset(X_train, y_train)
test_ds = TensorDataset(X_test, y_test)

train_loader = DataLoader(train_ds, batch_size=64, shuffle=True)
test_loader = DataLoader(test_ds, batch_size=64, shuffle=False)


In [27]:
class AmountAutoencoder(nn.Module):
    def __init__(self, input_dim):
        super().__init__()
        self.encoder = nn.Sequential(
            nn.Linear(input_dim, 512),
            nn.ReLU(),
            nn.Linear(512, 256),
            nn.ReLU(),
            nn.Linear(256, 128),
            nn.ReLU()
        )
        self.decoder = nn.Sequential(
            nn.Linear(128, 256),
            nn.ReLU(),
            nn.Linear(256, 512),
            nn.ReLU(),
            nn.Linear(512, 1)  # Output log(amount)
        )
    def forward(self, x):
        return self.decoder(self.encoder(x))


In [29]:
from tqdm import tqdm

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = AmountAutoencoder(input_dim=X.shape[1])

# Multi-GPU support
if torch.cuda.device_count() > 1:
    print(f"Using {torch.cuda.device_count()} GPUs")
    model = nn.DataParallel(model)

model = model.to(device)
criterion = nn.MSELoss()
optimizer = torch.optim.Adam(model.parameters(), lr=1e-4)

epochs = 2
for epoch in range(epochs):
    model.train()
    total_loss = 0

    # Wrap train_loader with tqdm for a progress bar
    loop = tqdm(train_loader, desc=f"Epoch {epoch+1}/{epochs}", leave=False)
    for xb, yb in loop:
        xb, yb = xb.to(device), yb.to(device)

        optimizer.zero_grad()
        outputs = model(xb)
        loss = criterion(outputs, yb)
        loss.backward()
        optimizer.step()

        total_loss += loss.item() * xb.size(0)
        loop.set_postfix(loss=loss.item())

    avg_loss = total_loss / len(train_loader.dataset)
    print(f"Epoch [{epoch+1}/{epochs}], Avg Loss: {avg_loss:.6f}")


Using 8 GPUs


Epoch [1/2], Avg Loss: 1.482090


Epoch [2/2], Avg Loss: 1.291470


In [31]:
model.eval()
with torch.no_grad():
    X_all = X_tensor.to(device)
    y_all = y_tensor.to(device)
    preds = model(X_all).cpu().numpy()

# Reconstruction errors
errors = np.abs(y_log.flatten() - preds.flatten())

# Threshold at 95th percentile
threshold = np.percentile(errors, 95)

df['reconstruction_error'] = errors
df['is_anomaly'] = df['reconstruction_error'] > threshold

df[['amount', 'reconstruction_error', 'is_anomaly']].head()


OutOfMemoryError: CUDA out of memory. Tried to allocate 107.94 GiB. GPU 0 has a total capacity of 139.72 GiB of which 26.90 GiB is free. Including non-PyTorch memory, this process has 109.74 GiB memory in use. Process 2630197 has 624.00 MiB memory in use. Process 2796816 has 2.45 GiB memory in use. Of the allocated memory 108.10 GiB is allocated by PyTorch, and 47.03 MiB is reserved by PyTorch but unallocated. If reserved but unallocated memory is large try setting PYTORCH_CUDA_ALLOC_CONF=expandable_segments:True to avoid fragmentation.  See documentation for Memory Management  (https://pytorch.org/docs/stable/notes/cuda.html#environment-variables)

In [33]:
model.eval()
preds_list = []

# Large batch size
test_loader = DataLoader(test_ds, batch_size=1024000, shuffle=False)

with torch.no_grad():
    for xb, _ in tqdm(test_loader, desc="Predicting batches"):
        xb = xb.to(device)
        preds_batch = model(xb).cpu()  # move back to CPU immediately
        preds_list.append(preds_batch)

# Combine all batch predictions
preds = torch.cat(preds_list, dim=0).numpy()

# Compute reconstruction errors
errors = np.abs(y_test.numpy().ravel() - preds.ravel())
threshold = np.percentile(errors, 95)


Predicting batches: 100%|██████████| 3/3 [00:38<00:00, 12.73s/it]


In [42]:
# Get original indices of test set
_, test_idx = train_test_split(
    np.arange(len(df)),
    test_size=0.2,
    random_state=42
)

# Map errors and anomalies back to the original DataFrame
df_test = df.iloc[test_idx].copy()
df_test['reconstruction_error'] = errors
df_test['is_anomaly'] = df_test['reconstruction_error'] > threshold
df_test

,FISCAL YEAR,FISCAL PERIOD,DEPARTMENT,DIVISION,VENDOR,BUDGET_REF,FUND_TYPE,FUND,FUND_DESCR,DEPTID,...,check_month,check_dayofweek,check_year_scaled,check_month_sin,check_month_cos,check_dow_sin,check_dow_cos,amount,reconstruction_error,is_anomaly
1354032,2021,8,DEPT OF HEALTH & SOCIAL SVS,MEDICAID & MEDICAL ASSISTANCE,OTHER EMPLOYMENT COSTS BENEFITS,2021.0,FEDERAL,225,Federal Funds,350201,...,2,4,-0.863661,8.660254e-01,0.500000,-0.433884,-0.900969,318.90,0.738336,False
361703,2021,10,DEPT OF HEALTH & SOCIAL SVS,MEDICAID & MEDICAL ASSISTANCE,DIV OF ACCOUNTING,2021.0,FEDERAL,225,Federal Funds,350201,...,4,0,-0.863661,8.660254e-01,-0.500000,0.000000,1.000000,37.46,0.572149,False
5665681,2023,4,DEPT OF EDUCATION,CHRISTINA SCHOOL DISTRICT,Delaware Assoc of School Admin,NaN,SPECIAL,100,General Fund,953300,...,10,2,-0.083571,-8.660254e-01,0.500000,0.974928,-0.222521,11.24,1.039839,False
7864920,2024,11,DELAWARE NATIONAL GUARD,DELAWARE NATIONAL GUARD,ARTESIAN WATER INC,2024.0,FEDERAL,225,Federal Funds,760101,...,5,2,1.476610,5.000000e-01,-0.866025,0.974928,-0.222521,189.07,0.715463,False
1014795,2021,1,DEPT OF HEALTH & SOCIAL SVS,DELAWARE PSYCHIATRIC CENTER,Local 1029,NaN,SPECIAL,100,General Fund,350630,...,7,0,-1.643752,-5.000000e-01,-0.866025,0.000000,1.000000,76.00,0.009689,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8598526,2024,11,DEPT OF EDUCATION,CHRISTINA SCHOOL DISTRICT,EMPLOYEE COMPENSATION,2024.0,FEDERAL,225,Federal Funds,953300,...,5,4,1.476610,5.000000e-01,-0.866025,-0.433884,-0.900969,2882.46,0.531902,False
5918170,2023,7,DEPT OF EDUCATION,SEAFORD SCHOOL DISTRICT,EMPLOYEE COMPENSATION,2023.0,GENERAL,100,General Fund,952300,...,1,4,0.696519,5.000000e-01,0.866025,-0.433884,-0.900969,79.00,2.210406,False
1113431,2021,12,DEPT OF EDUCATION,RED CLAY CONSOLIDATED SCH DIST,OTHER EMPLOYMENT COSTS BENEFITS,2021.0,GENERAL,100,General Fund,953200,...,6,3,-0.863661,1.224647e-16,-1.000000,0.433884,-0.900969,136.42,1.089864,False
5102033,2022,2,DEPT OF TRANSPORTATION,CAPITAL PROJECT COST CENTER,WW GRAINGER INC,2016.0,CAPITAL,301,DelDOT Capital,550500,...,8,1,-0.863661,-8.660254e-01,-0.500000,0.781831,0.623490,275.29,1.125848,False


In [40]:
# Show results
# Show only anomalies in the test set
df_test_anomalies = df_test[df_test['is_anomaly']]

# Display the relevant columns
# Sort anomalies by reconstruction error (highest first)
df_test_anomalies_sorted = df_test_anomalies.sort_values(
    by='reconstruction_error',
    ascending=False
)

# Display top anomalies
df_test_anomalies_sorted[['amount', 'reconstruction_error', 'is_anomaly']]

,amount,reconstruction_error,is_anomaly
2630966,1.50,13.091682,True
5206596,56.33,12.753857,True
5277288,3.00,12.039203,True
9000592,4.80,12.035208,True
3609964,15.60,11.272756,True
...,...,...,...
5715472,1038471.22,2.402217,True
9032818,4.41,2.402216,True
7381608,7806.01,2.402205,True
10327293,29389.00,2.402195,True


In [41]:
df.iloc[2630966]

FISCAL YEAR                                     2022
FISCAL PERIOD                                      9
DEPARTMENT             DEPARTMENT OF HUMAN RESOURCES
DIVISION              DIVISION OF STATEWIDE BENEFITS
VENDOR                              CVS PHARMACY INC
BUDGET_REF                                    2022.0
FUND_TYPE                                    SPECIAL
FUND                                             100
FUND_DESCR                              General Fund
DEPTID                                        160501
DEPTID_DESCR          DIVISION OF STATEWIDE BENEFITS
OP_UNIT                                          NaN
OP_UNIT_DESCR                                    NaN
APPROP                                           NaN
APPROP_DESCR                                 missing
ACCOUNT                                        21553
ACCOUNT_DESCR                     MEDICAL - EMPLOYER
PROGRAM                                        10EGW
PROGRAM_DESCR             Employer Group Waive

In [43]:
# Save the anomalies DataFrame to CSV
df_test_anomalies_sorted.to_csv('test_anomalies.csv', index=False)

In [44]:
df_test_anomalies_sorted.columns

Index(['FISCAL YEAR', 'FISCAL PERIOD', 'DEPARTMENT', 'DIVISION', 'VENDOR',
       'BUDGET_REF', 'FUND_TYPE', 'FUND', 'FUND_DESCR', 'DEPTID',
       'DEPTID_DESCR', 'OP_UNIT', 'OP_UNIT_DESCR', 'APPROP', 'APPROP_DESCR',
       'ACCOUNT', 'ACCOUNT_DESCR', 'PROGRAM', 'PROGRAM_DESCR', 'SCH_CODE',
       'SCH_CODE_DESCR', 'PC_BU', 'PC_BU_DESCR', 'PROJECT', 'PROJECT_DESCR',
       'ACTIVITY', 'ACTIVITY_DESCR', 'CHECK_NUMBER', 'CHECK_DATE', 'AMOUNT',
       'fiscal_year', 'fiscal_year_scaled', 'fp_sin', 'fp_cos', 'check_date',
       'check_year', 'check_month', 'check_dayofweek', 'check_year_scaled',
       'check_month_sin', 'check_month_cos', 'check_dow_sin', 'check_dow_cos',
       'amount', 'reconstruction_error', 'is_anomaly'],
      dtype='object')

In [51]:
# Remove 'is_anomaly' column
df_to_save = df_test_anomalies_sorted.drop(columns=['fiscal_year', 'check_date', 'amount'])

# Save to CSV
df_to_save.to_csv('test_anomalies_drop_1.csv', index=False)

In [49]:
df_to_save.columns

Index(['FISCAL YEAR', 'FISCAL PERIOD', 'DEPARTMENT', 'DIVISION', 'VENDOR',
       'BUDGET_REF', 'FUND_TYPE', 'FUND', 'FUND_DESCR', 'DEPTID',
       'DEPTID_DESCR', 'OP_UNIT', 'OP_UNIT_DESCR', 'APPROP', 'APPROP_DESCR',
       'ACCOUNT', 'ACCOUNT_DESCR', 'PROGRAM', 'PROGRAM_DESCR', 'SCH_CODE',
       'SCH_CODE_DESCR', 'PC_BU', 'PC_BU_DESCR', 'PROJECT', 'PROJECT_DESCR',
       'ACTIVITY', 'ACTIVITY_DESCR', 'CHECK_NUMBER', 'CHECK_DATE', 'AMOUNT',
       'fiscal_year_scaled', 'fp_sin', 'fp_cos', 'check_year', 'check_month',
       'check_dayofweek', 'check_year_scaled', 'check_month_sin',
       'check_month_cos', 'check_dow_sin', 'check_dow_cos',
       'reconstruction_error', 'is_anomaly'],
      dtype='object')

In [52]:
df_to_save

,FISCAL YEAR,FISCAL PERIOD,DEPARTMENT,DIVISION,VENDOR,BUDGET_REF,FUND_TYPE,FUND,FUND_DESCR,DEPTID,...,check_year,check_month,check_dayofweek,check_year_scaled,check_month_sin,check_month_cos,check_dow_sin,check_dow_cos,reconstruction_error,is_anomaly
2630966,2022,9,DEPARTMENT OF HUMAN RESOURCES,DIVISION OF STATEWIDE BENEFITS,CVS PHARMACY INC,2022.0,SPECIAL,100,General Fund,160501,...,2022,3,3,-0.083571,1.000000,6.123234e-17,0.433884,-0.900969,13.091682,True
5206596,2023,9,DEPARTMENT OF HUMAN RESOURCES,DIVISION OF STATEWIDE BENEFITS,Beverly Devine,2023.0,SPECIAL,100,General Fund,160501,...,2023,3,2,0.696519,1.000000,6.123234e-17,0.974928,-0.222521,12.753857,True
5277288,2023,5,DEPARTMENT OF HUMAN RESOURCES,DIVISION OF STATEWIDE BENEFITS,CVS PHARMACY INC,2023.0,SPECIAL,100,General Fund,160501,...,2022,11,3,-0.083571,-0.500000,8.660254e-01,0.433884,-0.900969,12.039203,True
9000592,2024,1,DEPARTMENT OF HUMAN RESOURCES,DIVISION OF STATEWIDE BENEFITS,MEDCO HEALTH SOLUTIONS INC,2024.0,SPECIAL,100,General Fund,160501,...,2023,7,3,0.696519,-0.500000,-8.660254e-01,0.433884,-0.900969,12.035208,True
3609964,2022,11,DEPARTMENT OF HUMAN RESOURCES,DIVISION OF STATEWIDE BENEFITS,MEDCO HEALTH SOLUTIONS INC,2022.0,SPECIAL,100,General Fund,160501,...,2022,5,1,-0.083571,0.500000,-8.660254e-01,0.781831,0.623490,11.272756,True
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5715472,2023,9,LEGAL,DEPARTMENT OF JUSTICE,EMPLOYEE COMPENSATION,2023.0,GENERAL,100,General Fund,150101,...,2023,3,4,0.696519,1.000000,6.123234e-17,-0.433884,-0.900969,2.402217,True
9032818,2024,11,DEPT OF EDUCATION,CHRISTINA SCHOOL DISTRICT,New Jersey Family Support,NaN,SPECIAL,100,General Fund,953300,...,2024,5,2,1.476610,0.500000,-8.660254e-01,0.974928,-0.222521,2.402216,True
7381608,2023,2,DEPT OF EDUCATION,CAESAR RODNEY SCHOOL DISTRICT,OTHER EMPLOYMENT COSTS BENEFITS,2023.0,GENERAL,100,General Fund,951000,...,2022,8,4,-0.083571,-0.866025,-5.000000e-01,-0.433884,-0.900969,2.402205,True
10327293,2024,1,DEPT OF TRANSPORTATION,MAINTENANCE DISTRICTS,PRIORITY SERVICES LLC,2023.0,SPECIAL,101,DelDOT Operational,550470,...,2023,7,1,0.696519,-0.500000,-8.660254e-01,0.781831,0.623490,2.402195,True
